In [96]:
import pandas as pd

# 파일 경로
csv_path = "/home/food/people/minju/final/100/labase_result.csv"

# CSV 읽기
df = pd.read_csv(csv_path, encoding="utf-8-sig")   #


In [97]:
(df['target_found']==True).sum()

82

In [98]:
df['target_rank'].sum()

152.0

In [99]:
df['target_rank'].mean()

1.853658536585366

In [100]:
mrr = (1 / df['target_rank']).mean()
print("MRR:", mrr)

MRR: 0.7941347270615563


In [101]:
import pandas as pd

# 파일 경로 설정
qa_result_path = "/home/food/people/minju/final/original_bm25.csv"

original_result_path = "/home/food/people/minju2/bm25_results.csv"

# CSV 파일 불러오기
df_qa = pd.read_csv(qa_result_path)
df_orig = pd.read_csv(original_result_path)

# 비교할 문서 열 이름
doc_cols = [f"문서{i}" for i in range(1, 11)]

# 결과 저장용 리스트
diff_rows = []

# 행별로 비교
for idx in range(min(len(df_qa), len(df_orig))):
    row_diff = {"id": idx, "diff_cols": []}
    for col in doc_cols:
        doc_qa = str(df_qa.loc[idx, col]).strip()
        doc_orig = str(df_orig.loc[idx, col]).strip()
        if doc_qa != doc_orig:
            row_diff["diff_cols"].append(col)
    
    if row_diff["diff_cols"]:
        diff_rows.append(row_diff)

# 결과 출력
print(f"총 {len(diff_rows)}개의 행에서 문서 차이 발생")
for diff in diff_rows:
    print(f"Row {diff['id']} → 차이 있는 문서: {', '.join(diff['diff_cols'])}")


총 21개의 행에서 문서 차이 발생
Row 0 → 차이 있는 문서: 문서5
Row 1 → 차이 있는 문서: 문서8
Row 2 → 차이 있는 문서: 문서4
Row 3 → 차이 있는 문서: 문서2, 문서5, 문서8, 문서9, 문서10
Row 4 → 차이 있는 문서: 문서1, 문서3, 문서5, 문서8, 문서9, 문서10
Row 5 → 차이 있는 문서: 문서2, 문서5, 문서9
Row 7 → 차이 있는 문서: 문서1, 문서3
Row 11 → 차이 있는 문서: 문서1
Row 12 → 차이 있는 문서: 문서1
Row 13 → 차이 있는 문서: 문서1, 문서2
Row 14 → 차이 있는 문서: 문서10
Row 16 → 차이 있는 문서: 문서4, 문서5
Row 19 → 차이 있는 문서: 문서1, 문서2, 문서3, 문서4, 문서6, 문서9, 문서10
Row 20 → 차이 있는 문서: 문서2, 문서5, 문서6, 문서8, 문서9, 문서10
Row 27 → 차이 있는 문서: 문서1
Row 29 → 차이 있는 문서: 문서1, 문서5, 문서9, 문서10
Row 31 → 차이 있는 문서: 문서3
Row 32 → 차이 있는 문서: 문서3, 문서4
Row 35 → 차이 있는 문서: 문서6
Row 36 → 차이 있는 문서: 문서1
Row 37 → 차이 있는 문서: 문서1


In [102]:
import pandas as pd
import re

# 파일 경로 설정
qa_result_path = "/home/food/people/minju/final/retriever_only_result.csv"
original_result_path = "/home/food/people/minju/final/original_hybrid_real.csv"

# CSV 파일 불러오기
df_qa = pd.read_csv(qa_result_path)
df_orig = pd.read_csv(original_result_path)

# 비교할 문서 열 이름
doc_cols = [f"문서{i}" for i in range(1, 11)]

# 문자열 전처리 + 토큰 분할 함수
def normalize(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)  # 특수문자 제거
    text = re.sub(r"\s+", " ", text)     # 여러 공백 -> 하나
    return set(text.strip().split())     # 단어 집합으로 반환

# 유사도 함수 (Jaccard index 기반)
def jaccard_sim(set1, set2):
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

# 결과 저장용 리스트
diff_rows = []
SIM_THRESHOLD = 0.85  # 유사도 임계값 (1에 가까울수록 엄격함)

# 행별로 비교
for idx in range(min(len(df_qa), len(df_orig))):
    row_diff = {"id": idx, "diff_cols": []}
    for col in doc_cols:
        set_qa = normalize(df_qa.loc[idx, col])
        set_orig = normalize(df_orig.loc[idx, col])
        sim = jaccard_sim(set_qa, set_orig)

        if sim < SIM_THRESHOLD:
            row_diff["diff_cols"].append(f"{col} ({sim:.2f})")

    if row_diff["diff_cols"]:
        diff_rows.append(row_diff)

# 결과 출력
print(f"총 {len(diff_rows)}개의 행에서 문서 차이 발생")
for diff in diff_rows:
    print(f"Row {diff['id']} → 차이 있는 문서: {', '.join(diff['diff_cols'])}")


총 4개의 행에서 문서 차이 발생
Row 2 → 차이 있는 문서: 문서3 (0.02), 문서4 (0.01), 문서5 (0.18), 문서6 (0.03), 문서7 (0.04), 문서8 (0.01), 문서9 (0.01), 문서10 (0.00)
Row 3 → 차이 있는 문서: 문서3 (0.10), 문서4 (0.06), 문서5 (0.05), 문서6 (0.04), 문서7 (0.12), 문서8 (0.05), 문서9 (0.07), 문서10 (0.00)
Row 15 → 차이 있는 문서: 문서3 (0.09), 문서4 (0.11), 문서5 (0.22), 문서6 (0.01), 문서7 (0.01), 문서8 (0.09), 문서9 (0.03), 문서10 (0.00)
Row 25 → 차이 있는 문서: 문서7 (0.06), 문서8 (0.00), 문서9 (0.00)
